# 1조 인공지능을 활용한 마스크 착용 감지 모델 (고윤홍, 박신, 안혜빈)

## 프로젝트를 하게 된 계기

 - 전국적으로 확진이 다시 늘어나고 있는 추세이고 마스크 미착용자에 대한 처벌이 강화되고 있습니다. 하지만 마스크를 올바른 방법으로 착용한 사람들 사이에 그렇지 않은 사람들이 존재합니다. 그런 이들을 통해 COVID19의 감염이 전파되는 것을 예방하고 마스크 착용에 대한 감시역이 없더라도 마스크 착용을 권장하는 인공지능을 만들어보고 싶어서 이 프로젝트를 하게 되었습니다.

## 프로젝트 개요

 - 기존의 마스크 착용 감지 인공지능을 이용하여 실시간 영상 속에서 인원 수를 화면에 표시하고 각각의 마스크 착용 비율을 표시하며 마스크를 착용하지 않은 사람이 섞여있는 경우를 탐지할 수 있는 인공지능을 목표로 하고 있습니다. 

## 인공지능 학습 단계

#### 첫번째 단계: 모듈 불러오기

 **사용된 모듈**
```pyrthon
 - keras             # tensorflow 위에서 작동하며 더 상위 수준의 기능을 제공
 - sklearn           # machine learning의 분석에서 중요
 - imutils           # Open CV가 처리하기 힘든 이미지나 비디오 스트림 파일의 처리를 보완
 - matplotlib        # 파이썬에서 그래프 표시
 - numpy             # 저 수준의 언어로 파이썬의 리스트보다 효율적
 - argparse          # 다양한 인자 관리
 - os                # 컴퓨터의 디렉토리, 파일을 이용가능하게 만드는 기능
 ```

```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os 
```

#### 두번째 단계: 외부 데이터 불러오기

```python
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True, 
                help="path to input dataset")                    # --dataset 데이터 불러오기
ap.add_argument("-p", "--plot", type=str, default="plot.png", 
                help="path to output loss/accuracy plot")        # --plot의 데이터를 불러와 plot.png에 저장
ap.add_argument("-m", "--model", type=str, default="mask_detector.model", 
                help="path to output face mask detector model")  # --model의 데이터를 불러와 mask_detector.model에 저장
args = vars(ap.parse_args())                                     # args에 각 데이터들을 업데이트
```

#### 세번째 단계: 인공지능 학습에 필요한 설정

```python
INIT_LR = 1e-4     # 학습률: 0.0001
EPOCHS = 20        # 에폭
BS = 32            # 배치 크기

print("[INFO] loading images...") 
imagePaths = list(paths.list_images(args["dataset"]))      # dataset의 이미지 데이터들을 배열로 만들기
data = []          # 문제의 보기와 같은 역할을 할 배열
labels = []        # 문제의 정답과 같은 역할을 할 배열

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)

# 각각의 배열을 넘파이 배열로 만들기
data = np.array(data, dtype="float32")
labels = np.array(labels) 

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels) 

# data와 lables배열을 나누기
(trainX, testX, trainY, testY) = train_test_split(data, labels, 
                                                  test_size=0.20, stratify=labels, random_state=42) 

# 이미지 데이터 전처리 과정
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")
```

#### 네번째 단계:  모델 설정

```python
#모바일 환경에서 작동가능하도록 최적화
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False
```

#### 다섯번째 단계: 학습 시작

```python
# 모델 가져오기
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# 학습
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# 예측값 구하기 및 출력
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=lb.classes_))

# mask_detector.model에 저장
print("[INFO] saving mask detector model...")
model.save(args["model"], save_format="h5")
```

#### 여섯번째 단계: 시각화

```python
# plot.png에 저장
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])
```

## 영상을 통한 데이터 검증

#### 1) Import packages

```python
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
```

#### 2) Argument setting

```python
ap = argparse.ArgumentParser()   #  --를 인식함.    외부 데이터 불러들이기

ap.add_argument("-f", -face"-", type=str,   #외부에서  --face, --model, 그리고 --confidence의 데이터를 받아옴.
    default="face_detector",
    help="path to face detector model directory")    #--face의 데이터는 face_detector에 추가합니다.
    
ap.add_argument("-m", "--model", type=str,
    default="mask_detector.model",
    help="path to trained face mask detector model")      #--model의 데이터는 mask_detector.model에 추가합니다.
    
ap.add_argument("-c", "--confidence", type=float, default=0.5,
    help="minimum probability to filter weak detections")    
    
args = vars(ap.parse_args())   #마지막으로 args에 각각의 데이터들을 업데이트해줍니다
```

#### 3) detect_and_predict_mask 함수 정의

```python
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]   #frame배열의 1,2번째 index값을 h(세로),w(가로)에 저장   #frame은 실시간 비디오의 화면
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),(104.0, 177.0, 123.0))  #이미지 가공해서 blob에 저장

    faceNet.setInput(blob)  #blob에 저장된 가공한 이미지를 faceNet에 저장
    detections = faceNet.forward()  #순방향으로 딥러닝 네트워크를 실행해 detection에 저장

    faces = []  #얼굴을 모으는 리스트
    locs = []   #box의 좌표를 모으는 리스트
    preds = []  #마스크 예측한 것을 모으는 리스트
    
    for i in range(0, detections.shape[2]):  #200번 반복
        confidence = detections[0, 0, i, 2]  #얼굴을 인식하고 마스크를 착용했는지에 대한 추측 정확도를 confidence에 저장
                                              #detections는 실시간으로 찍히는 화면이 저장되는 것

        if confidence > args["confidence"]:   #기본값이 0.5로 설정. confidence가 크면 마스크를 잘 쓴 것.
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h]   #detection에서 얼굴크기를 이용해 box의 네 모퉁이를 box에 저장.
            (startX, startY, endX, endY) = box.astype("int")  #box의 두 모퉁이 값을 정수로 바꿔서 좌표로 지정
            (startX, startY) = (max(0, startX), max(0, startY))   #좌표가 음수가 안나오게 하기 위함.
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))   #box가 화면 밖으로 안나가게 하기 위함.

            face = frame[startY:endY, startX:endX]   #frame(배경+사람)에서 box크기 만클을 slicing 한 것을 face에 저장.
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))   
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)    #가공된 face가 faces에 추가됨.
            locs.append((startX, startY, endX, endY))   #locs에는 box의 좌표가 추가됨.

    if len(faces) > 0:   #저장된 얼굴의 개수가 1개 이상이면 얼굴 사진들을 넘파이 어레이로 만듦.
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)   #예측한 것(마스크를 착용했을 확률과 착용하지 않았을 확률)  

    return  (locs,preds)  #box의 좌표들과 예측한 값을 return
```

#### 4) face_detector폴더 내의 파일들을 불러와 faceNet에 저장

```python
print("[INFO] loading face detector model...")

prototxtPath = os.path.sep.join([args["face"], "deploy.prototxt"])   #"deploy.prototxt"파일을 불러와서 prototxtPath에 저장 

weightsPath = os.path.sep.join([args["face"],"res10_300x300_ssd_iter_140000.caffemodel"])

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)  #dnn(딥러닝 네트워크)를 사용해 prototxtPath와 weightPath를 faceNet에 저장
```

#### 5) mask_detector.model을 불러와 maskNet에 저장, 비디오 시작

```python
print("[INFO] loading face mask detector model...")
maskNet = load_model(args["model"])      #mask_detector.model을 불러와서 model에 저장하고, 그 model을 maskNet이라는 변수에 저장.

print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()     #video stream을 초기화하고, 카메라 센서를 warm up시킴.
time.sleep(2.0)
```

#### 6) 실시간 video를 읽어오고, q키를 누르면 video stop

```python
while True:

    frame = vs.read()  #실시간 화면이 frame
    frame = imutils.resize(frame, width=1100)  #frame크기 조정

    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)   #frame(화면)과 4,5번에서 얻은 faceNet,maskNet을 함수에 통과시킴.

    maskCount = 0     #추가한 부분
    nonMaskCount = 0  #추가한 부분    #실시간으로 사람 수를 세기 위해 매번 초기화함. 
    for (box, pred) in zip(locs, preds):    #box의 좌표들이 있는 locs변수와 예측한 것이 모여있는 pred변수에서 각각 하나씩 세트로 보냄. 
        (startX, startY, endX, endY) = box   #box의 2점의 좌표
        (mask, withoutMask) = pred   #마스크를 착용했을 확률, 착용하지 않았을 확률을 pred에 저장

        if mask > withoutMask: #추가한 부분  #반복문에 들어온 사람이 마스크를 착용했을 확률이 더 높다면 
            maskCount += 1     #추가한 부분
        else:                  #추가한 부분
            nonMaskCount += 1  #추가한 부분  #이렇게 매 순간 마스크를 착용한 사람과 착용하지 않을 사람의 수를 헤아린다. 

        label = "Mask" if mask > withoutMask else "No Mask"  #마스크를 착용했을 확률이 높다면 label에 Mask, 아니면 No Mask
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)   #전자의 경우라면 초록색으로, 후자라면 빨간색으로 글자를 표시한다.
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100) 
        label2 = "Good Job" if mask > withoutMask else "Warning[Wear Mask]"  #추가한 부분  #착용 여부에 대한 반응을 해줌.
        
        cv2.putText(frame, label, (startX, startY - 10)   #frame에 위의 label꾸며서 보이게 함.,                           
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)     
        cv2.putText(frame, label2, (startX, endY + 20),   #추가한 부분  #frame에 위의 label2를 꾸며서 출력
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)      
        cv2.rectangle(frame, (startX, startY), (endX, endY),color, 2)   #frame에 꾸민 box를 올림.

    color2 = (0, 255, 0) if nonMaskCount == 0 else (0, 0, 255)    #추가한 부분  #한 명이라도 마스크를 착용하지 않았으면 빨간색으로 color지정
    label3 = "{}/{}".format(maskCount, nonMaskCount + maskCount)  #추가한 부분  #몇 명 중, 몇 명이 마스크를 착용했는지에 대한 확률 계산
    cv2.putText(frame, label3, (100, 130),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color2, 2)  #추가한 부분  #frame위에 color2색을 이용해 label3을 보여줌.

    label4 = "# of Wearing Masks/# of Detected"   #추가한 부분
    cv2.putText(frame, label4, (100, 100),        #추가한 부분
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 2)   #frame위에 label4를 보여줌.
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF   #사용자가 종료할 때 까지 기다림.

    if key == ord("q"):
        break   #사용자가 'q'키를 누르면 실시간 비디오 종료된. 
        
cv2.destroyAllWindows()

vs.stop()   
```

## 데모 가이드 문서

#### 1) 프로젝트 다운로드

아래 명령어를 실행하여 프로젝트를 다운로드한다.
```
$ git clone https://github.com/younhong/COVID19-Mask-Detection-Model
```


> 만약 git이 설치되어 있지 않다면 [링크](https://github.com/younhong/COVID19-Mask-Detection-Model)로 들어가서 수동으로 zip 파일을 다운로드

#### 2) 환경 설정

환경설정: 가상환경을 설정   
Why?
> 가상환경을 설치하지 않으면 기존의 파이썬 설치된 환경과 충돌하여 의도치 않은 에러가 날 수 있다.

##### Step1) 가상환경 생성에 필요한 패키지 설치
Mac의 경우
```
$ pip install virtualenv virtualenvwrapper
```

Window의 경우
```
$ pip install virtualenv virtualenvwrapper-win
```

##### Step2) 환경변수 설정
Mac의 경우
- 아래 명령어로 virtualenvwrapper.sh가 설치된 위치 확인(결과값을 기억해둘 것)
```
$ find / -name virtualenvwrapper.sh 2>/dev/null
```
> ![Result](snapshot/path.png)
> (경로가 두 가지 이상 나오면 둘 중에 하나 선택해서 복사)

- 아래 명령어로 .zshrc 파일을 열고 환경 변수를 추가(아래 이미지 마지막 3줄 참고)   
```
$ vi ~/.zshrc
``` 
> 마지막 줄의 source 뒤의 경로를 위에서 체크한 경로로 변경
> ![Result](snapshot/zshrc.png)

- 수정한 내용 적용
```
$ source ~/.zshrc
```

- 실행 후 재부팅

Window의 경우
- 아래 명령어로 환경변수 경로를 프로젝트 경로로 설정
```
$ setx WORKON_HOME="프로젝트가 설치된 경로"  
```
- 재부팅

##### Step3) 가상환경 생성
```
$ mkvirtualenv test(test 대신 다른 이름 써도 괜찮음)
```
> ![Result](snapshot/mkvirtualenv.png)

> (맥) 이미지 맨 아래에 표시된 부분과 같이 설정한 환경 이름이 표시되면 성공

> (윈도우) ls를 터미널에서 입력 후 설정한 환경명이 폴더로 생성되었다면 성공

##### Step4) 설치된 가상환경에 프로젝트 실행에 필요한 파이썬 모듈 설치
```
$ pip install -r requirements.txt
```

#### 3) 데이터 학습 단계

```
$ python train_mask_detector.py --dataset dataset
```

> dataset 폴더 안에 있는 3800개의 이미지를 가져야 20번의 epoch 동안 각각 96번의 step을 거치며 학습을 진행

#### 4) 데이터 학습 결과에 대한 추가 설명

![Image](snapshot/result-report.png)

![Image](plot.png)

![Image](snapshot/precision-recall-f1-accuracy.png) 

- (편의상 TP, TN, FP, FN으로 표시합니다)

##### 4-1) Precision
> 모델이 True라고 예측한 값 중 실제 결과가 True의 값을 가지는 답의 비율

\begin{align}
\frac{TP}{(TP + FP)}
\end{align}

##### 4-2) Recall
> 실제 True인 값 중 모델이 True라고 옳게 예측한 답의 비율

\begin{align}
\frac{TP}{(TP + FN)}
\end{align}

##### 4-3) F1
> Precision과 Recall의 조화평균

\begin{align}
\frac{2 * precision * recall}{(precision + recall)}
\end{align}

##### 4-4) Accuracy
> True라고 옳게 예측한 것과 False라고 옮게 예측한 것의 비율

\begin{align}
\frac{TP + TN}{(TP + FP + TN + FN)}
\end{align}

##### 4-5) 자료 출처
> [출처](https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/)

#### 5) 데이터 검증 단계

실시간 웹캠 영상을 통한 검증

```
$ python detect_mask_video.py
```

## 보완할 점

#### 1) 한정된 데이터
- 현재 학습에 사용한 건 3800개의 이미지. 모든 상황을 커버하기는 부족하다. 더 정확한 검증을 위해서는 더 많은 학습 데이터가 필요하다.

#### 2) 마스크 불량 착용자 감지
- 이 프로젝트는 마스크 착용 여부만 감지한다. 마스크를 코 끝까지 올리지 않은 사람들을 모두 감지하고 싶다면 그 부분에 해당하는 데이터와 레이블이 필요하다. 기존에 학습에 사용한 데이터만큼 많은 수의 데이터가 주어진다면 가능할 것으로 판단된다.

#### 3) 관리자에게 알림
- 현재는 마스크 불량자를 감지하고 경고를 주는 것에 끝난다. 자동으로 관리자에게 메시지를 보낼 수 있도록 보완하면 좋을 것 같다.

## 기대효과

- 종교시설 등 집단 감염의 위험이 있는 장소에서 마스크 미착용자를 감지하여 건물의 출입이나 이용을 제한하거나 격리 조취를 취할 수 있다. 

- 다수의 인원이 감지되더라도 실시간으로 마스크 미착용자의 수를 구분해낼 수 있다.